In [ ]:
!pip install Elasticsearch
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.9/431.9 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 2.5 MB/s eta 0:00:00


In [ ]:
from elasticsearch import Elasticsearch
import datetime
import json
import numpy as np
from sentence_transformers import SentenceTransformer

In [ ]:
API_KEY = 'dy1fbU5ZMEJMWm03TmRtVUFNdHU6dmtPUEhPTDJUdXk3ZkFueFppc0daQQ=='
CLOUD_ID = '59edb5339821479e902881bf2403e966:dXMtY2VudHJhbDEuZ2NwLmNsb3VkLmVzLmlvJGY4ZWIzZmNhZGY2ZDQ0NTFhY2EwOGEyYzZmZDc2ZWY4JDY3MmI3MTViMmE4ZjQ1ZTk4MzcyMzFkOGZiYjYyMWE2'

In [ ]:
es = Elasticsearch(cloud_id=CLOUD_ID, api_key=API_KEY)
es.info()

ObjectApiResponse({'name': 'instance-0000000001', 'cluster_name': 'f8eb3fcadf6d4451aca08a2c6fd76ef8', 'cluster_uuid': 'Y-6Ru43iREOjWpIdGKH2qw', 'version': {'number': '8.12.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '1665f706fd9354802c02146c1e6b5c0fbcddfbc9', 'build_date': '2024-01-11T10:05:27.953830042Z', 'build_snapshot': False, 'lucene_version': '9.9.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [ ]:
index_name = "ir-hw4-data"

In [ ]:
index_settings = {
    "index.mapping.total_fields.limit": 2000
}

es.indices.create(index=index_name, body={"settings": index_settings})

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'ir-hw4-data'})

In [ ]:
with open('revised_data.json') as f:
    data = json.load(f)

In [ ]:
for item in data:
    for item_key, item_data in item.items():
        es.index(index=index_name, id=item_data['ID'], body=item_data)

In [ ]:
response = es.search(index=index_name, body={"query": {"match_all": {}}, "size": 100})

# Print the total number of documents and some information about each document
print(f"Total documents in the index: {response['hits']['total']['value']}")
for hit in response['hits']['hits']:
    print(f"Document ID: {hit['_id']}, Source: {hit['_source']}")

Total documents in the index: 100
Document ID: 1, Source: {'ID': '1', 'link': 'https://www.amazon.com/Over-Ear-Headphones-Adjustable-Closed-Back-Audiophiles/dp/B06XKJ2GK4/ref=sr_1_217?c=ts&keywords=Over-Ear+Headphones&qid=1703082644&s=aht&sr=1-217&ts_id=12097479011', 'name': 'Meze 99 Neo | Wired Closed-Back Headset for Audiophiles | Gaming | Podcasts | Home Office | Over-Ear Headphones with Mic and Self Adjustable Headband', 'price': '199', 'description': "The uptown audiophile's delight: 99 NEO closed-back headphones offer a modern twist to the iconic 99 series, blending Meze Audio's signature sound and design. With coal black textured earcups and cast zinc alloy hardware, the wired over-ear 99 Neo headset exudes a sleek, contemporary aesthetic.\nPOWER EFFICIENT WITH ANY DEVICE due to low impedance (26 Ohm). No need for amplification making them suitable for gaming, home office, studio recording, video editing, and podcast monitoring. They easily plug-in with your computer, laptop, sm

In [ ]:
import random

random_item = random.choice(data)

item_key, item_data = list(random_item.items())[0]
query_detail = item_data['detail']

query = {
    "query": {
        "simple_query_string": {
            "fields": ["detail"],
            "query": query_detail,
            "default_operator": "or"
        }
    },
    "size": 20
}

results = es.search(index=index_name, body=query)

print("random_item: \n",random_item)

for hit in results['hits']['hits']:
    similar_item = hit['_source']
    score = hit['_score']
    print(f"Score: {score}, Item: {similar_item}")

random_item: 
 {'item5': {'ID': '5', 'link': 'https://www.amazon.com/Skullcandy-Over-Ear-Headphones-Microphone-Bluetooth/dp/B0CBLM9MMC/ref=sr_1_221?c=ts&keywords=Over-Ear+Headphones&qid=1703082644&s=aht&sr=1-221&ts_id=12097479011', 'name': 'Skullcandy Crusher Evo Over-Ear Wireless Headphones with Sensory Bass, 40 Hr Battery, Microphone, Works with iPhone Android and Bluetooth Devices - True Black', 'price': '149', 'description': 'What is Crusher Tech? Boost your bass with the sensory bass slider, mellow out or go all in skull crushing! Crusher Tech enhances your iPhone, Android or Computer audio experience\nFeel the bass tuned to you - Listen to songs the way they were made to be heard. With the Personal Sound feature, the Crusher Evo analyzes your hearing and produces sound specifically for YOU\nOne fat battery with 40 hours of playtime. And with Rapid Charge technology, you get 4 hours of charge in 10 minutes with the included USB-C cable. Alternatively, stay connected at all times w

In [ ]:
item_ids_range = range(1, 21)

item_ids_query = " OR ".join([f"ID:{item_id}" for item_id in item_ids_range])

matrix_size = 20
recommender_system_matrix = np.zeros((matrix_size, matrix_size), dtype=int)

for row_item_id in item_ids_range:

    random_item = data[row_item_id-1]
    item_key, item_data = list(random_item.items())[0]
    query_detail = item_data['detail']
    row_item_query = {
        "query": {
            "bool": {
                "must": [
                    {
                        "simple_query_string": {
                            "fields": ["detail"],
                            "query": query_detail,
                            "default_operator": "or"
                        }
                    }
                ],
                "filter": {
                    "query_string": {
                        "query": item_ids_query
                    }
                }
            }
        },
        "size": 20
    }

    row_results = es.search(index=index_name, body=row_item_query)

    row_scores = [(hit['_id'], hit['_score']) for hit in row_results['hits']['hits']]

    row_scores.sort(key=lambda x: x[1], reverse=True)
    row_ids = [(hit['_id']) for hit in row_results['hits']['hits']]

    for rank, (col_item_id, _) in enumerate(row_scores):
        col_index = item_ids_range.index(int(col_item_id))
        recommender_system_matrix[row_item_id - 1, col_index] = rank + 1

print(recommender_system_matrix)


[[ 1 14  7  4 18  9 12 17  5 20 10 15  8  2 13 11  3 16  6 19]
 [18  1 12  5 10 11  3  6  8 17 15  2  7 20  9 13 14 16  4 19]
 [17 13  1  3 15 11  5  9  8 18  7 10  6 19  2 12 14 16  4 20]
 [15  5  3  1 17 10  4 14  2 18 11  7 12 19  6 13 16  9  8 20]
 [17  4  5  8  1 18  9  3 12 20  6  2  7 19 11 10 15 16 13 14]
 [14 10  5  3 18  1 15 12  7 16  2 11  9 17  8  4 13 19  6 20]
 [18  3  6  4 15 16  1 11  5 17  9  2 12 19  7 14 13 10  8 20]
 [19  2  3 11  8 10  4  1 17  6 15  7 12 18  9  5 14 16 13 20]
 [14  7  5  2 18 11  4 17  1 16  9  6  8 20  3 15 13 12 10 19]
 [17 13  8  5 19 14  6  2  4  1 12  3  9 20  7 11 16 15 10 18]
 [16 13  3  4 14  2  7 15  6 19  1  5  8 18 11 12 10 17  9 20]
 [19  2 10  6 15 16  3 13  7 14  8  1  9 18  4 17 11 12  5 20]
 [19  7  3  6 16 11  9 13  8 17 10  4  1 20  5 14 15 12  2 18]
 [ 2 13  6 10 16 12 11 15 19 20  8  9  5  1 18  7  3 17  4 14]
 [19  8  6  4 17 12  5 14  2 15  7  3 11 20  1  9 18 13 10 16]
 [14  8  3  5 10  2  7  6  9 18  4 11 12 16 13  1 17 19

In [ ]:
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Step 1: Load JSON data
with open('revised_data.json', 'r') as file:
    data = json.load(file)

# Step 2: Preprocess Text Data
def preprocess_text(text):
    # Add more preprocessing steps if needed
    return text.lower()

# Step 3: Extract Details for the first 20 items
details = []

for item in data[:20]:  # Process only the first 20 items
    for item_key, item_value in item.items():
        details.append(preprocess_text(item_value.get('detail', '')))

# Step 4: Calculate Cosine Similarity
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(details)
similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
# print(similarity_matrix)
# Print the similarity scores for the first 20 items
for i in range(len(details)):
    for j in range(i + 1, len(details)):
        print(f"Similarity between item {i + 1} and item {j + 1}: {similarity_matrix[i][j]}")


Similarity between item 1 and item 2: 0.17594528179208524
Similarity between item 1 and item 3: 0.2043794352933015
Similarity between item 1 and item 4: 0.23739813966717818
Similarity between item 1 and item 5: 0.17126652324493477
Similarity between item 1 and item 6: 0.17323701798783478
Similarity between item 1 and item 7: 0.19378051689309336
Similarity between item 1 and item 8: 0.12789097890775364
Similarity between item 1 and item 9: 0.21965843595946155
Similarity between item 1 and item 10: 0.14768777570119382
Similarity between item 1 and item 11: 0.1938926851135487
Similarity between item 1 and item 12: 0.15918976113669375
Similarity between item 1 and item 13: 0.1503460274028314
Similarity between item 1 and item 14: 0.5127760574258445
Similarity between item 1 and item 15: 0.17126447162790764
Similarity between item 1 and item 16: 0.1377717367938832
Similarity between item 1 and item 17: 0.23152286451788767
Similarity between item 1 and item 18: 0.14707533020569186
Similarity

In [ ]:
ground_truth = [[1, 13, 7, 4, 12, 10, 9, 19, 5, 17, 8, 14, 15, 2, 11, 18, 3, 16, 6, 20],
                [17, 1, 6, 4, 11, 13, 3, 7, 14, 18, 8, 2, 12, 15, 9, 19, 10, 20, 5, 16],
                [17, 8, 1, 3, 14, 10, 6, 13, 7, 15, 4, 11, 9, 19, 2, 16, 12, 18, 5, 20],
                [16, 7, 2, 1, 14, 9, 4, 13, 3, 15, 6, 10, 11, 19, 5, 17, 18, 12, 8, 20],
                [16, 6, 2, 8, 1, 18, 3, 7, 9, 13, 4, 11, 14, 20, 5, 17, 10, 15, 12, 19],
                [14, 6, 4, 3, 17, 1, 10, 11, 5, 15, 2, 12, 13, 19, 8, 7, 9, 18, 16, 20],
                [18, 2, 5, 3, 9, 13, 1, 11, 4, 14, 7, 6, 15, 20, 8, 17, 10, 16, 12, 19],
                [18, 4, 2, 5, 7, 12, 3, 1, 10, 6, 11, 14, 16, 19, 9, 15, 13, 17, 8, 20],
                [18, 8, 4, 2, 13, 11, 5, 15, 1, 16, 7, 10, 12, 20, 3, 17, 9, 14, 6, 19],
                [17, 12, 3, 6, 13, 15, 7, 2, 5, 1, 8, 9, 14, 20, 4, 16, 10, 18, 11, 19],
                [16, 7, 4, 2, 10, 3, 11, 15, 5, 20, 1, 9, 14, 18, 6, 13, 8, 17, 12, 19],
                [16, 2, 7, 4, 12, 15, 3, 11, 8, 18, 9, 1, 13, 19, 5, 17, 10, 14, 6, 20],
                [12, 8, 3, 6, 17, 11, 9, 15, 5, 18, 7, 16, 1, 19, 4, 13, 14, 10, 2, 20],
                [2, 15, 3, 6, 11, 12, 8, 17, 14, 20, 7, 9, 10, 1, 16, 13, 4, 18, 5, 19],
                [16, 19, 2, 4, 12, 10, 6, 11, 3, 13, 5, 7, 14, 20, 1, 15, 9, 17, 8, 18],
                [12, 10, 2, 5, 14, 4, 9, 16, 6, 20, 3, 13, 17, 19, 7, 1, 15, 11, 8, 18],
                [10, 12, 3, 9, 14, 8, 13, 15, 2, 18, 4, 11, 6, 19, 5, 20, 1, 16, 7, 17],
                [15, 8, 4, 2, 12, 18, 6, 13, 3, 16, 5, 11, 7, 17, 9, 14, 10, 1, 20, 19],
                [16, 4, 2, 3, 14, 12, 5, 13, 6, 15, 8, 8, 7, 19, 10, 17, 11, 18, 1, 20],
                [15, 4, 5, 8, 11, 16, 2, 18, 3, 14, 6, 17, 12, 20, 7, 13, 9, 19, 10, 1],
               ]

# Create a NumPy array from your numbers
ground_truth = np.array(ground_truth)

In [ ]:
predictions = [
        [1, 14, 7, 4, 18, 9, 12, 17, 5, 20, 10, 15, 8, 2, 13, 11, 3, 16, 6, 19],
        [18, 1, 12, 5, 10, 11, 3, 6, 8, 17, 15, 2, 7, 20, 9, 13, 14, 16, 4, 19],
        [17, 13, 1, 3, 15, 11, 5, 9, 8, 18, 7, 10, 6, 19, 2, 12, 14, 16, 4, 20],
        [15, 5, 3, 1, 17, 10, 4, 14, 2, 18, 11, 7, 12, 19, 6, 13, 16, 9, 8, 20],
        [17, 4, 5, 8, 1, 18, 9, 3, 12, 20, 6, 2, 7, 19, 11, 10, 15, 16, 13, 14],
        [14, 10, 5, 3, 18, 1, 15, 12, 7, 16, 2, 11, 9, 17, 8, 4, 13, 19, 6, 20],
        [18, 3, 6, 4, 15, 16, 1, 11, 5, 17, 9, 2, 12, 19, 7, 14, 13, 10, 8, 20],
        [19, 2, 3, 11, 8, 10, 4, 1, 17, 6, 15, 7, 12, 18, 9, 5, 14, 16, 13, 20],
        [14, 7, 5, 2, 18, 11, 4, 17, 1, 16, 9, 6, 8, 20, 3, 15, 13, 12, 10, 19],
        [17, 13, 8, 5, 19, 14, 6, 2, 4, 1, 12, 3, 9, 20, 7, 11, 16, 15, 10, 18],
        [16, 13, 3, 4, 14, 2, 7, 15, 6, 19, 1, 5, 8, 18, 11, 12, 10, 17, 9, 20],
        [19, 2, 10, 6, 15, 16, 3, 13, 7, 14, 8, 1, 9, 18, 4, 17, 11, 12, 5, 20],
        [19, 7, 3, 6, 16, 11, 9, 13, 8, 17, 10, 4, 1, 20, 5, 14, 15, 12, 2, 18],
        [2, 13, 6, 10, 16, 12, 11, 15, 19, 20, 8, 9, 5, 1, 18, 7, 3, 17, 4, 14],
        [19, 8, 6, 4, 17, 12, 5, 14, 2, 15, 7, 3, 11, 20, 1, 9, 18, 13, 10, 16],
        [14, 8, 3, 5, 10, 2, 7, 6, 9, 18, 4, 11, 12, 16, 13, 1, 17, 19, 15, 20],
        [10, 4, 2, 11, 18, 9, 5, 7, 12, 19, 3, 6, 15, 14, 8, 17, 1, 16, 13, 20],
        [14, 8, 7, 2, 15, 19, 3, 9, 5, 17, 12, 4, 6, 18, 10, 16, 11, 1, 13, 20],
        [12, 4, 3, 6, 19, 11, 7, 13, 8, 18, 10, 5, 2, 14, 9, 17, 15, 16, 1, 20],
        [11, 14, 4, 16, 6, 17, 3, 18, 5, 19, 8, 12, 2, 13, 10, 9, 15, 20, 7, 1],
]

predictions = np.array(predictions)

NDCG

In [ ]:
size = len(ground_truth[0])
DCG_gt = []
for row in ground_truth:
  sum = row[0]

  for i in range(1, size):
    res = row[i] / np.log2(i+1)
    sum += res

  DCG_gt.append(sum)

In [ ]:
DCG_gt

[74.41028571563906,
 76.01758857964805,
 79.43928221366528,
 77.30187756231246,
 76.8659474491475,
 76.19271869194478,
 75.86491415525184,
 76.00240547473794,
 80.98535070206023,
 82.964823980048,
 77.85868092984795,
 76.82261762098142,
 78.61396729667601,
 75.93865439306758,
 86.72915423288734,
 78.26665288529792,
 79.97638058778998,
 79.35334939713297,
 76.06257768016935,
 78.31269761848786]

In [ ]:
size = len(predictions[0])
DCG_pred = []

for row in predictions:
  sum = row[0]

  for i in range(1, size):
    res = row[i] / np.log2(i+1)
    sum += res

  DCG_pred.append(sum)

In [ ]:
DCG_pred

[76.40848049224975,
 78.53522105441023,
 83.33763023006365,
 76.45860123081053,
 77.8569087985149,
 80.52978488963836,
 78.92687177628633,
 77.39670667187227,
 78.40214086668246,
 86.07597088535354,
 82.46025270620211,
 81.18236286211666,
 82.71199444655124,
 78.03713283868409,
 83.512659399398,
 76.34596506688038,
 73.953942827932,
 80.1024149256786,
 75.52635311053932,
 83.81937448923696]

In [ ]:
# NDCG = DCG_pred / DCG_gt

NDCG = np.divide(DCG_pred, DCG_gt)

print(NDCG)

[1.02685374 1.03311908 1.0490733  0.98909113 1.01289207 1.05692232
 1.04036065 1.01834549 0.96810275 1.03749959 1.05910159 1.0567508
 1.05212849 1.02763386 0.96291334 0.97545969 0.9246973  1.00943962
 0.99295022 1.07031653]


In [ ]:
mean_NDCG = np.mean(NDCG)
print(mean_NDCG)

1.018182578109381


SPEARMAN Correlation Coefficient

In [ ]:
rank_differences = ground_truth - predictions

sum_of_squared_differences = np.sum(rank_differences**2, axis=1)

n = ground_truth.shape[1]

spearman_correlation_coefficients = 1 - (6 * sum_of_squared_differences) / (n * (n**2 - 1))
mean_spearman_correlation_coefficient = np.mean(spearman_correlation_coefficients)

In [ ]:
mean_spearman_correlation_coefficient

0.821766917293233

MAP

In [ ]:
import numpy as np

def precision_at_k(ground_truth, predictions, k):
    num_rows = len(predictions)
    precision_values = []

    for i in range(num_rows):
        ground_truth_row = ground_truth[i][:k]
        predicted_row = predictions[i][:k]
        common_elements = 0
        for j in range(k):
            if ground_truth_row[j] == predicted_row[j]:
                common_elements = common_elements + 1
        precision = common_elements / k
        precision_values.append(precision)

    return precision_values

def calculate_average_precision(ground_truth, predictions):
    num_rows = len(predictions)
    MAP_values = []
    for i in range(num_rows):
        ground_truth_row = ground_truth[i]
        predicted_row = predictions[i]
        counter = 0
        size = len(predicted_row)
        sum = 0
        for j in range(size):
            if predicted_row[j] == ground_truth_row[j]:
                counter = counter + 1
                precision_values = precision_at_k(ground_truth, predictions, j+1)
                sum = sum + precision_values[i]
        MAP = sum / counter
        MAP_values.append(MAP)
    return MAP_values

average_precisions = calculate_average_precision(ground_truth, predictions)
for i, avg_p in enumerate(average_precisions):
    print(f"Average Precision for row {i + 1}: {avg_p:.2f}")

Average Precision for row 1: 0.55
Average Precision for row 2: 0.33
Average Precision for row 3: 0.56
Average Precision for row 4: 0.24
Average Precision for row 5: 0.38
Average Precision for row 6: 0.50
Average Precision for row 7: 0.55
Average Precision for row 8: 0.18
Average Precision for row 9: 0.35
Average Precision for row 10: 0.46
Average Precision for row 11: 0.42
Average Precision for row 12: 0.31
Average Precision for row 13: 0.43
Average Precision for row 14: 0.46
Average Precision for row 15: 0.20
Average Precision for row 16: 0.19
Average Precision for row 17: 0.43
Average Precision for row 18: 0.39
Average Precision for row 19: 0.28
Average Precision for row 20: 0.11


In [ ]:
MAP = np.mean(average_precisions)
print(MAP)

0.3655846308036158


#Phase2

In [ ]:
question = input("Enter a Question: ")

print("You entered:", question)

#sample question: how much is the price of Silensys E7 Active Noise Cancelling Bluetooth Headphones?


Enter a Question: which headphone comes with the best warranty?
You entered: which headphone comes with the best warranty?


In [ ]:
query = {
    "query": {
        "simple_query_string": {
            "query": question,
            "default_operator": "or"
        }
    },
}

results = es.search(index=index_name, body=query)

for hit in results['hits']['hits']:
    similar_item = hit['_source']
    score = hit['_score']
    print(f"Score: {score}, Item: {similar_item}")

Score: 16.481297, Item: {'ID': '79', 'link': 'https://www.amazon.com/PowerLocus-Bluetooth-Headphones-Over-Black/dp/B09999JX2F/ref=sr_1_290?c=ts&keywords=Over-Ear+Headphones&qid=1703082748&s=aht&sr=1-290&ts_id=12097479011', 'name': 'PowerLocus Bluetooth Headphones Over Ear, [Bass-Mode Button] Wireless Headphones, Foldable Hi-Fi Stereo, Soft Memory Foam Earmuffs, Metal Extendable Sides, Headset with Microphone for Phone/PC/TV', 'price': '19', 'description': '[HUGE BATTERY PLAYTIME & AUX MODE] – Finally bluetooth headphones with battery life enough to support you listening audio and watching movies for many weeks to come. Don’t worry if your battery is 0%, the over-ear headphones have wired mode available anytime. Probably the best wireless headphone for travelling. Get them back for less than 1 hour with Fast Charging USB-C technology.\n[PROBABLY THE MOST COMFORTABLE] – Foldable headphone with space saving design which you can take anywhere with the package included transport carry case.

In [ ]:
first_item = results['hits']['hits'][0]['_source']

item_string = json.dumps(first_item, ensure_ascii=False)

chunk_size = 800
overlap = 100

chunks = [item_string[i:i+chunk_size] for i in range(0, len(item_string), chunk_size - overlap)]

chunks_data = [{"id": str(i+1), "description_chunk": chunk} for i, chunk in enumerate(chunks)]

with open('chunks_output.json', 'w') as f:
    json.dump(chunks_data, f, ensure_ascii=False, indent=2)


In [ ]:
new_index_name = "chunk_data_7"

In [ ]:
with open('chunks_output.json') as f:
    chunks_data = json.load(f)

for chunk in chunks_data:
    es.index(index=new_index_name, body=chunk)

In [ ]:
response = es.search(index=new_index_name, body={"query": {"match_all": {}}, "size": 100})

# Print the total number of documents and some information about each document
print(f"Total documents in the index: {response['hits']['total']['value']}")
for hit in response['hits']['hits']:
    print(f"Document ID: {hit['_id']}, Source: {hit['_source']}")

Total documents in the index: 11
Document ID: g4a2ZY0BmSmlXLSSHpER, Source: {'id': '1', 'description_chunk': '{"ID": "79", "link": "https://www.amazon.com/PowerLocus-Bluetooth-Headphones-Over-Black/dp/B09999JX2F/ref=sr_1_290?c=ts&keywords=Over-Ear+Headphones&qid=1703082748&s=aht&sr=1-290&ts_id=12097479011", "name": "PowerLocus Bluetooth Headphones Over Ear, [Bass-Mode Button] Wireless Headphones, Foldable Hi-Fi Stereo, Soft Memory Foam Earmuffs, Metal Extendable Sides, Headset with Microphone for Phone/PC/TV", "price": "19", "description": "[HUGE BATTERY PLAYTIME & AUX MODE] – Finally bluetooth headphones with battery life enough to support you listening audio and watching movies for many weeks to come. Don’t worry if your battery is 0%, the over-ear headphones have wired mode available anytime. Probably the best wireless headphone for travelling. Get them back for less than 1 hour with Fast Charg'}
Document ID: QfC2ZY0BLZm7NdmUHimS, Source: {'id': '2', 'description_chunk': 'bably the 

In [ ]:
results = es.search(index=new_index_name, body=query)

for hit in results['hits']['hits']:
    similar_item = hit['_source']
    score = hit['_score']
    print(f"Score: {score}, Item: {similar_item}")

Score: 3.842785, Item: {'id': '11', 'description_chunk': 'ver feel the embarrassment of not having smooth communication during Zoom or phone calls. With upgraded Bluetooth 5.0 chip, this wireless headphone will pair instantly with Phones, Tablets, Laptops, PC and TV. Use them also as gaming headphones and enjoy perfect privacy while watching movies with the low latency technology.\\n[Backed UP with 2 Years WARRANTY] – Unlike others which offer you only 1 Year Warranty and slower support, By choosing PowerLocus, You are covered with 2 Years PowerLocus warranty and 100% Customer satisfaction – 24/7 Customer Support. Simply get in touch with us via amazon and we will solve your problem within less than 2 hours."}'}
Score: 3.6422558, Item: {'id': '3', 'description_chunk': 'nd without any distortion even at the loudest volume levels with these over ear wireless headphones. Deep bass button is available, extended treble and clean highs result in balanced audio which will spoil your ears. HD 

In [ ]:
first_4_items = results['hits']['hits'][:2]

description_chunks = [hit['_source']['description_chunk'] for hit in first_4_items]

concatenated_result = " ".join(description_chunks)

# Create the prompt
prompt = f"Based on this information: {concatenated_result} , Answer this question: {question}"

print(prompt)


Based on this information: ver feel the embarrassment of not having smooth communication during Zoom or phone calls. With upgraded Bluetooth 5.0 chip, this wireless headphone will pair instantly with Phones, Tablets, Laptops, PC and TV. Use them also as gaming headphones and enjoy perfect privacy while watching movies with the low latency technology.\n[Backed UP with 2 Years WARRANTY] – Unlike others which offer you only 1 Year Warranty and slower support, By choosing PowerLocus, You are covered with 2 Years PowerLocus warranty and 100% Customer satisfaction – 24/7 Customer Support. Simply get in touch with us via amazon and we will solve your problem within less than 2 hours."} nd without any distortion even at the loudest volume levels with these over ear wireless headphones. Deep bass button is available, extended treble and clean highs result in balanced audio which will spoil your ears. HD stereo sound will make watching movies and series a thrilling experience.\n[BE THE KING/QUEE

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!git clone https://github.com/google/sentencepiece.git
!cd sentencepiece
!mkdir build
!cd build
!cmake ..
!make -j $(nproc)
!sudo make install
!sudo ldconfig -v

In [ ]:
import torch
print(torch.cuda.current_device())

0


In [ ]:
!pip install optimum
!pip install auto-gptq
!pip install transformers

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install accelerate

In [ ]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4


In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

In [ ]:
from huggingface_hub import hf_hub_download
from llama_cpp import Llama

In [ ]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

In [ ]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=2048, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32,
    n_ctx=2048

    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [ ]:
lcpp_llm.params.n_gpu_layers

32

In [ ]:
# prompt = my_text

prompt = question
system_prompt = concatenated_result

prompt_template=f'''SYSTEM: {system_prompt}.

USER:  {prompt}

ASSISTANT:
'''

In [ ]:
response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

Llama.generate: prefix-match hit


In [ ]:
print(response["choices"][0]["text"])

SYSTEM: ver feel the embarrassment of not having smooth communication during Zoom or phone calls. With upgraded Bluetooth 5.0 chip, this wireless headphone will pair instantly with Phones, Tablets, Laptops, PC and TV. Use them also as gaming headphones and enjoy perfect privacy while watching movies with the low latency technology.\n[Backed UP with 2 Years WARRANTY] – Unlike others which offer you only 1 Year Warranty and slower support, By choosing PowerLocus, You are covered with 2 Years PowerLocus warranty and 100% Customer satisfaction – 24/7 Customer Support. Simply get in touch with us via amazon and we will solve your problem within less than 2 hours."} nd without any distortion even at the loudest volume levels with these over ear wireless headphones. Deep bass button is available, extended treble and clean highs result in balanced audio which will spoil your ears. HD stereo sound will make watching movies and series a thrilling experience.\n[BE THE KING/QUEEN OF THE ONLINE MEE

#Phase3(Extra part)

In [ ]:
index_name_extra = "extra_part12"

In [ ]:
# Create an index with a mapping for dense_vector field
es.indices.create(index=index_name_extra, ignore=400, body={
  "mappings": {
    "properties": {
      "chunk_embedding": {
        "type": "dense_vector",
        "dims": 384  # Adjust based on your model's embedding size, e.g., 384 for MiniLM L6
      }
    }
  }
})

<ipython-input-6-ed54e92e4cdb>:2: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index=index_name_extra, ignore=400, body={


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'extra_part12'})

In [ ]:
with open('revised_data.json', 'r') as file:
    data = json.load(file)

In [ ]:
item_string = json.dumps(data, ensure_ascii=False)
chunk_size = 500
overlap = 0
chunks = [item_string[i:i+chunk_size] for i in range(0, len(item_string), chunk_size - overlap)]

In [ ]:
for chunk in chunks:
  print(chunk)

[{"item1": {"ID": "1", "link": "https://www.amazon.com/Over-Ear-Headphones-Adjustable-Closed-Back-Audiophiles/dp/B06XKJ2GK4/ref=sr_1_217?c=ts&keywords=Over-Ear+Headphones&qid=1703082644&s=aht&sr=1-217&ts_id=12097479011", "name": "Meze 99 Neo | Wired Closed-Back Headset for Audiophiles | Gaming | Podcasts | Home Office | Over-Ear Headphones with Mic and Self Adjustable Headband", "price": "199", "description": "The uptown audiophile's delight: 99 NEO closed-back headphones offer a modern twist to
 the iconic 99 series, blending Meze Audio's signature sound and design. With coal black textured earcups and cast zinc alloy hardware, the wired over-ear 99 Neo headset exudes a sleek, contemporary aesthetic.\nPOWER EFFICIENT WITH ANY DEVICE due to low impedance (26 Ohm). No need for amplification making them suitable for gaming, home office, studio recording, video editing, and podcast monitoring. They easily plug-in with your computer, laptop, smartphone, tablet MP3 & MP4 player. Using it wi

In [ ]:
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')  # You can choose a different model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.69k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/314 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
for chunk in chunks:
    # Get the embedding for the entire chunk
    embedding = model.encode(chunk)
    # Index the representation in Elasticsearch with the KNN field
    es.index(index=index_name_extra, body={'chunk_embedding': embedding.tolist(), 'original_chunk':chunk})

In [ ]:
response = es.search(index=index_name_extra, body={"query": {"match_all": {}}, "size": 100})

# Print the total number of documents and some information about each document
print(f"Total documents in the index: {response['hits']['total']['value']}")
for hit in response['hits']['hits']:
    print(f"Document ID: {hit['_id']}, Source: {hit['_source']}")

Total documents in the index: 1622
Document ID: 8IZXaI0BmSmlXLSSx5ky, Source: {'chunk_embedding': [-0.07247529923915863, 0.022062331438064575, 0.1413012444972992, -0.2400815635919571, 0.016471682116389275, -0.3040430545806885, 0.48274606466293335, 0.021335404366254807, 0.23135459423065186, -0.031146708875894547, 0.5133547186851501, -0.1627686619758606, -0.11341594904661179, -0.14620402455329895, -0.018931003287434578, 0.015919633209705353, 0.5239194631576538, 0.45885753631591797, -0.07269026339054108, 0.0037755537778139114, 0.8417681455612183, -0.07436338067054749, -0.09889596700668335, 0.12897181510925293, 0.009438816457986832, 0.3281922936439514, -0.18188458681106567, -0.05273851379752159, 0.02507421374320984, -0.20373307168483734, 0.06805828213691711, 0.5982645750045776, 0.2245585024356842, 0.22368639707565308, 0.06343759596347809, -0.1599423885345459, 0.08499227464199066, -0.2304106205701828, 0.01648784428834915, 0.12362579256296158, -0.3600916266441345, -0.305511474609375, -0.2895

In [ ]:
user_input = input("Enter your question: ")

# Get the embedding for the user input
user_question_embedding = model.encode(user_input)

#sample question: how much is the price of Silensys E7 Active Noise Cancelling Bluetooth Headphones?

Enter your question: how much is the price of Silensys E7 Active Noise Cancelling Bluetooth Headphones?


In [ ]:
response = es.search(index=index_name_extra, body={
  "query": {
    "script_score": {
      "query": {"match_all": {}},
      "script": {
        "source": "cosineSimilarity(params.query_vector, 'chunk_embedding') + 1.0",
        "params": {"query_vector": user_question_embedding.tolist()}
      }
    }
  }
})

for hit in response['hits']['hits']:
    # Get the original chunk associated with the hit
    original_chunk = hit['_source']['original_chunk']

    # Decode or directly use the original chunk data
    print(original_chunk)

dering their price range, these headphones are on par with other options available in the $30-$50 range.\n\nIn conclusion, if you're seeking a budget-friendly pair of headphones that offer decent audio quality without breaking the bank, these headphones are a reasonable choice. Just keep in mind the minor inconvenience of sound pauses during app switching.", "They're comfortable and last all week with their battery life. Good value for the price and they look nice. The low end of the sound isn't
out these headphones is it CAN be wired, which I love.\n\nThe sound quality is amazing for the price. It blows my wireless earbuds out of the water in terms out sound, from vocal clarity to bass to every little sound that goes into the song. They can get quite loud and powerful. The noise cancelation also seems to work quite well.\n\nI've been wearing these pretty much non-stop since they arrived. They're pretty light but also decently durable for the price, so you really can't tell you've got 